Compare de-prod-etl-proj-ya5acmz0.claritin_event_lake.events_tbl with markets.ODL_execution_stage.

Using notebook due to BQ perimeter limitation on projects.

In [299]:
import pandas as pd
import numpy as np
import pyrds
from pyrds.data import gbq_query

PROJECT = 'de-prod-warehouse-evdz5h3o'
CORE_PROJECT = "dsr-core-proj-rcjtxz9f"

Read Claritin dataset

In [278]:
sql = '''
    select timestamp,
        JSON_EXTRACT_SCALAR(data, '$.orderId') as order_id,
        JSON_EXTRACT_SCALAR(data_rows, '$.trade_id') as trade_id,
        upper(JSON_EXTRACT_SCALAR(data, '$.exchange')) as exchange,
        upper(JSON_EXTRACT_SCALAR(data, '$.orderDetails.base')) as base,
        upper(JSON_EXTRACT_SCALAR(data, '$.orderDetails.counter')) as counter,
        JSON_EXTRACT(data_rows, '$.price') as price,
        JSON_EXTRACT(data_rows, '$.amount') as amount,
        JSON_EXTRACT(data_rows, '$.proceeds') as proceeds,
        upper(JSON_EXTRACT_SCALAR(data_rows, '$.side')) as side,
        from
        de-prod-etl-proj-ya5acmz0.claritin_event_lake.events_tbl
        ,UNNEST (JSON_EXTRACT_ARRAY(data, '$.orderTrades')) as data_rows
        where schemaType='OrderTradesData' and date(timestamp) <= DATE_SUB(current_date(), INTERVAL 1 DAY) 
        order by timestamp
'''
df_claritin = gbq_query(sql, project=PROJECT)
df_claritin

,timestamp,order_id,trade_id,exchange,base,counter,price,amount,proceeds,side
0,2020-06-25 21:18:23.639000+00:00,1246351047847936,116245841,BITSTAMP,XRP,EUR,0.16283,2682.32,16464.84063133,BUY
1,2020-06-25 21:18:34.201000+00:00,1246351091208192,116245861,BITSTAMP,XRP,USD,0.18235,289.89999987,52.86326498,SELL
2,2020-06-25 21:18:34.201000+00:00,1246351091208192,116245862,BITSTAMP,XRP,USD,0.18228,8229.09809,1499.99999985,SELL
3,2020-06-25 21:18:34.201000+00:00,1246351091208192,116245863,BITSTAMP,XRP,USD,0.18228,7945.84254113,1448.3681784,SELL
4,2020-06-25 21:21:26.196000+00:00,1246351795908608,116245954,BITSTAMP,XRP,USD,0.18259,135.82840385,743.515,BUY
...,...,...,...,...,...,...,...,...,...,...
87399,2020-07-20 23:53:43.947000+00:00,Q5xemviEwytFzCRI,16559633,BITSO,XRP,MXN,4.38,38333.84957319,167902.26113057,SELL
87400,2020-07-20 23:55:22.990000+00:00,1255236989399041,117629660,BITSTAMP,XRP,USD,0.19496,78.044,400.0,BUY
87401,2020-07-20 23:55:22.990000+00:00,1255236989399041,117629661,BITSTAMP,XRP,USD,0.19496,1119.656,5738.95157981,BUY
87402,2020-07-20 23:56:56.024000+00:00,81206d75d8044265a43e084a3d6afa48,81206d75d8044265a43e084a3d6afa48,COINSPH,XRP,PHP,9.58,6145.347599,58872.43,SELL


Read ODL_executions_stage table starting from the earlist timestamp in Claritin dataset

In [283]:
earlist_claritin_ts = df_claritin.iloc[0]['timestamp']
sql = '''
    SELECT execution_time as timestamp,order_id,execution_id as trade_id,upper(exchange) as exchange,
    upper(base) as base,upper(counter) as counter,
    price,size,upper(side) as side FROM `dsr-core-proj-rcjtxz9f.markets.ODL_executions_stage`
    where execution_time >= '{}' and date(execution_time) <= DATE_SUB(current_date(), INTERVAL 1 DAY) 
    order by execution_time
'''.format(str(earlist_claritin_ts))
df_oes = gbq_query(sql, project=CORE_PROJECT)
df_oes

,timestamp,order_id,trade_id,exchange,base,counter,price,size,side
0,2020-06-25 21:18:27.031000+00:00,61ac27aa85c0432fa6709da7981fb9e9,61ac27aa85c0432fa6709da7981fb9e9,COINSPH,XRP,,9.08000,2738.594714,SELL
1,2020-06-25 21:18:34.200000+00:00,1246351091208192,116245862,BITSTAMP,XRP,USD,0.18228,8229.098090,SELL
2,2020-06-25 21:18:34.200000+00:00,1246351091208192,116245863,BITSTAMP,XRP,USD,0.18228,7945.842541,SELL
3,2020-06-25 21:18:34.200000+00:00,1246351091208192,116245861,BITSTAMP,XRP,USD,0.18235,289.900000,SELL
4,2020-06-25 21:21:26.195000+00:00,1246351795908608,116245956,BITSTAMP,XRP,USD,0.18261,17413.622453,BUY
...,...,...,...,...,...,...,...,...,...
87389,2020-07-20 23:53:43.947000+00:00,Q5xemviEwytFzCRI,16559633,BITSO,XRP,MXN,4.38000,38333.849573,SELL
87390,2020-07-20 23:55:22.990000+00:00,1255236989399041,117629661,BITSTAMP,XRP,USD,0.19496,5738.951580,BUY
87391,2020-07-20 23:55:22.990000+00:00,1255236989399041,117629660,BITSTAMP,XRP,USD,0.19496,400.000000,BUY
87392,2020-07-20 23:56:56.024000+00:00,81206d75d8044265a43e084a3d6afa48,81206d75d8044265a43e084a3d6afa48,COINSPH,XRP,PHP,9.58000,6145.347599,SELL


Joined the 2 datasets

In [284]:
df_claritin_indexed = df_claritin.set_index(['order_id', 'trade_id'])
df_oes_indexed = df_oes.set_index(['order_id', 'trade_id'])
df_joined = df_claritin_indexed.join(df_oes_indexed, lsuffix='_claritin', rsuffix='_oes')
df_joined

timestamp_claritin  \
order_id                         trade_id                                                            
1246351047847936                 116245841                        2020-06-25 21:18:23.639000+00:00   
1246351091208192                 116245861                        2020-06-25 21:18:34.201000+00:00   
                                 116245862                        2020-06-25 21:18:34.201000+00:00   
                                 116245863                        2020-06-25 21:18:34.201000+00:00   
1246351795908608                 116245954                        2020-06-25 21:21:26.196000+00:00   
...                                                                                            ...   
Q5xemviEwytFzCRI                 16559633                         2020-07-20 23:53:43.947000+00:00   
1255236989399041                 117629660                        2020-07-20 23:55:22.990000+00:00   
                                 117629661                        2020-07-20 23:55:22.990000+00:00   
81206d75d8044265a43e084a3d6afa48 81206d75d8044265a43e084a3d6afa48 2020-07-20 23:56:56.024000+00:00   
6159521406                       6159521409                       2020-07-20 23:59:26.503000+00:00   

                                                                  exchange_claritin  \
order_id                         trade_id                                             
1246351047847936                 116245841                                 BITSTAMP   
1246351091208192                 116245861                                 BITSTAMP   
                                 116245862                                 BITSTAMP   
                                 116245863                                 BITSTAMP   
1246351795908608                 116245954                                 BITSTAMP   
...                                                                             ...   
Q5xemviEwytFzCRI                 16559633                                     BITSO   
1255236989399041                 117629660                                 BITSTAMP   
                                 117629661                                 BITSTAMP   
81206d75d8044265a43e084a3d6afa48 81206d75d8044265a43e084a3d6afa48           COINSPH   
6159521406                       6159521409                              BTCMARKETS   

                                                                  base_claritin  \
order_id                         trade_id                                         
1246351047847936                 116245841                                  XRP   
1246351091208192                 116245861                                  XRP   
                                 116245862                                  XRP   
                                 116245863                                  XRP   
1246351795908608                 116245954                                  XRP   
...                                                                         ...   
Q5xemviEwytFzCRI                 16559633                                   XRP   
1255236989399041                 117629660                                  XRP   
                                 117629661                                  XRP   
81206d75d8044265a43e084a3d6afa48 81206d75d8044265a43e084a3d6afa48           XRP   
6159521406                       6159521409                                 XRP   

                                                                  counter_claritin  \
order_id                         trade_id                                            
1246351047847936                 116245841                                     EUR   
1246351091208192                 116245861                                     USD   
                                 116245862                                     USD   
                                 116245863                                     USD   
1246351795908608         

Validate row level correctness

In [288]:
unmatch = []
threshold = 0.05
timestamp_threshold_in_seconds = 60

exact_match = ['exchange', 'base', 'counter', 'side']
for idx, row in df_joined.iterrows():
    err_items = []
    for _metric in exact_match:
        _val_left = row["{}_claritin".format(_metric)]
        _val_right = row["{}_oes".format(_metric)]
        if pd.isnull(_val_left) or pd.isnull(_val_right):
            err_items.append("item missing")
        elif _val_left != _val_right:
            err_items.append(_metric)
            
    # compare timestamp
    _second_diff = (row['timestamp_claritin'] - row['timestamp_oes']).total_seconds()
    if _second_diff > timestamp_threshold_in_seconds:
        err_items.append("timestamp")
    
    # check amount
    amount_oes = float(row["size"])
    if pd.isnull(amount_oes) or pd.isnull(row['side_oes']):
        continue
    amount_claritin = float(row["proceeds"] if row['side_oes'].upper() == 'BUY' else row["amount"])
    if ((amount_oes - amount_claritin) / amount_claritin >= threshold):
        err_items.append("amount")
        
    if len(err_items) > 0:
        _unmatch_row = list(row.values) + [",".join(err_items)]
        unmatch.append(_unmatch_row)

df_mismatch = pd.DataFrame(unmatch, columns=list(df_joined.columns) + ["unmatched_items"])
df_mismatch

,timestamp_claritin,exchange_claritin,base_claritin,counter_claritin,price_claritin,amount,proceeds,side_claritin,timestamp_oes,exchange_oes,base_oes,counter_oes,price_oes,size,side_oes,unmatched_items


No mismatch found. Take a look at missing values

In [286]:
print(df_joined.isna().sum())
df_missing = df_claritin_indexed[~df_claritin_indexed.isin(df_oes_indexed)].dropna()
df_missing

timestamp_claritin     0
exchange_claritin      0
base_claritin          0
counter_claritin       0
price_claritin         0
amount                 0
proceeds               0
side_claritin          0
timestamp_oes         12
exchange_oes          12
base_oes              12
counter_oes           12
price_oes             12
size                  12
side_oes              12
dtype: int64


timestamp  \
order_id                         trade_id                                                            
1246351047847936                 116245841                        2020-06-25 21:18:23.639000+00:00   
842f6691b2c04c50847e238bbfa724ea 842f6691b2c04c50847e238bbfa724ea 2020-06-26 13:59:59.972000+00:00   
6128626881                       6128626883                       2020-06-30 23:45:26.531000+00:00   
d4fbcc24a4184ffdaf1becfe29c8ce3b d4fbcc24a4184ffdaf1becfe29c8ce3b 2020-06-30 23:45:33.851000+00:00   
fea543347be9462191d03f8ad38fb90d fea543347be9462191d03f8ad38fb90d 2020-07-02 09:59:59.547000+00:00   
1253188481998848                 117347758                        2020-07-15 04:59:59.085000+00:00   
                                 117347759                        2020-07-15 04:59:59.085000+00:00   
1253262212714496                 117359795                        2020-07-15 09:59:59.679000+00:00   
                                 117359796                        2020-07-15 09:59:59.679000+00:00   
                                 117359797                        2020-07-15 09:59:59.679000+00:00   
a3Qi19GWe4nrftFV                 16547655                         2020-07-19 23:59:09.916000+00:00   
                                 16547656                         2020-07-19 23:59:09.916000+00:00   

                                                                     exchange  \
order_id                         trade_id                                       
1246351047847936                 116245841                           BITSTAMP   
842f6691b2c04c50847e238bbfa724ea 842f6691b2c04c50847e238bbfa724ea     COINSPH   
6128626881                       6128626883                        BTCMARKETS   
d4fbcc24a4184ffdaf1becfe29c8ce3b d4fbcc24a4184ffdaf1becfe29c8ce3b     COINSPH   
fea543347be9462191d03f8ad38fb90d fea543347be9462191d03f8ad38fb90d     COINSPH   
1253188481998848                 117347758                           BITSTAMP   
                                 117347759                           BITSTAMP   
1253262212714496                 117359795                           BITSTAMP   
                                 117359796                           BITSTAMP   
                                 117359797                           BITSTAMP   
a3Qi19GWe4nrftFV                 16547655                               BITSO   
                                 16547656                               BITSO   

                                                                  base  \
order_id                         trade_id                                
1246351047847936                 116245841                         XRP   
842f6691b2c04c50847e238bbfa724ea 842f6691b2c04c50847e238bbfa724ea  XRP   
6128626881                       6128626883                        XRP   
d4fbcc24a4184ffdaf1becfe29c8ce3b d4fbcc24a4184ffdaf1becfe29c8ce3b  XRP   
fea543347be9462191d03f8ad38fb90d fea543347be9462191d03f8ad38fb90d  XRP   
1253188481998848                 117347758                         XRP   
                                 117347759                         XRP   
1253262212714496                 117359795                         XRP   
                                 117359796                         XRP   
                                 117359797                         XRP   
a3Qi19GWe4nrftFV                 16547655                          XRP   
                                 16547656                          XRP   

                                                                  counter  \
order_id                         trade_id                                   
1246351047847936                 116245841                            EUR   
842f6691b2c04c50847e238bbfa724ea 842f6691b2c04c50847e238bbfa724ea     PHP   
6128626881                       6128626883                           AUD   
d4fbcc24a4184ffdaf1becfe29c8ce3b d4fbcc24a4184ffdaf1becfe29c8ce3b     PHP   
fea543347be9462191d03f8ad3

12 records from claritin_event_lake.events_tbl misses in ODL_execution_stage

Missing data distribution

In [297]:
df_missing['date'] = df_missing['timestamp'].dt.date
pd.DataFrame(df_missing.groupby('date')['timestamp'].count()).reset_index().rename(columns={"timestamp":"missing_cnt"})

,date,missing_cnt
0,2020-06-25,1
1,2020-06-26,1
2,2020-06-30,2
3,2020-07-02,1
4,2020-07-15,5
5,2020-07-19,2
